In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize

In [2]:
#literatürden alınan blok
# Utility functions for LorentzNet
def unsorted_segment_sum(data, segment_ids, num_segments):
    result = data.new_zeros((num_segments, data.size(1)))
    result.index_add_(0, segment_ids, data)
    return result

def unsorted_segment_mean(data, segment_ids, num_segments):
    result = data.new_zeros((num_segments, data.size(1)))
    count = data.new_zeros((num_segments, data.size(1)))
    result.index_add_(0, segment_ids, data)
    count.index_add_(0, segment_ids, torch.ones_like(data))
    return result / count.clamp(min=1)

def normsq4(p):
    psq = torch.pow(p, 2)
    return 2 * psq[..., 0] - psq.sum(dim=-1)

def dotsq4(p, q):
    psq = p * q
    return 2 * psq[..., 0] - psq.sum(dim=-1)

def psi(p):
    return torch.sign(p) * torch.log(torch.abs(p) + 1)

# LorentzNet Building Blocks
class LGEB(nn.Module):
    def __init__(self, n_input, n_output, n_hidden, n_node_attr=0,
                 dropout=0., c_weight=1.0, last_layer=False):
        super(LGEB, self).__init__()
        self.c_weight = c_weight
        n_edge_attr = 2  # dims for Minkowski norm & inner product

        self.phi_e = nn.Sequential(
            nn.Linear(n_input * 2 + n_edge_attr, n_hidden, bias=False),
            nn.BatchNorm1d(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU())

        self.phi_h = nn.Sequential(
            nn.Linear(n_hidden + n_input + n_node_attr, n_hidden),
            nn.BatchNorm1d(n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output))

        layer = nn.Linear(n_hidden, 1, bias=False)
        torch.nn.init.xavier_uniform_(layer.weight, gain=0.001)

        self.phi_x = nn.Sequential(
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            layer)

        self.phi_m = nn.Sequential(
            nn.Linear(n_hidden, 1),
            nn.Sigmoid())

        self.last_layer = last_layer
        if last_layer:
            del self.phi_x

    def m_model(self, hi, hj, norms, dots):
        out = torch.cat([hi, hj, norms, dots], dim=1)
        out = self.phi_e(out)
        w = self.phi_m(out)
        out = out * w
        return out

    def h_model(self, h, edges, m, node_attr):
        i, j = edges
        agg = unsorted_segment_sum(m, i, num_segments=h.size(0))
        agg = torch.cat([h, agg, node_attr], dim=1)
        out = h + self.phi_h(agg)
        return out

    def x_model(self, x, edges, x_diff, m):
        i, j = edges
        trans = x_diff * self.phi_x(m)
        trans = torch.clamp(trans, min=-100, max=100)
        agg = unsorted_segment_mean(trans, i, num_segments=x.size(0))
        x = x + agg * self.c_weight
        return x

    def minkowski_feats(self, edges, x):
        i, j = edges
        x_diff = x[i] - x[j]
        norms = normsq4(x_diff).unsqueeze(1)
        dots = dotsq4(x[i], x[j]).unsqueeze(1)
        norms, dots = psi(norms), psi(dots)
        return norms, dots, x_diff

    def forward(self, h, x, edges, node_attr=None):
        i, j = edges
        norms, dots, x_diff = self.minkowski_feats(edges, x)

        m = self.m_model(h[i], h[j], norms, dots)
        if not self.last_layer:
            x = self.x_model(x, edges, x_diff, m)
        h = self.h_model(h, edges, m, node_attr)
        return h, x, m
        

class LorentzNet(nn.Module):
    def __init__(self, n_scalar, n_hidden, n_class = 2, n_layers = 6, c_weight = 1e-3, dropout = 0.):
        super(LorentzNet, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.embedding = nn.Linear(n_scalar, n_hidden)
        self.LGEBs = nn.ModuleList([LGEB(self.n_hidden, self.n_hidden, self.n_hidden, 
                                    n_node_attr=n_scalar, dropout=dropout,
                                    c_weight=c_weight, last_layer=(i==n_layers-1))
                                    for i in range(n_layers)])
        self.graph_dec = nn.Sequential(nn.Linear(self.n_hidden, self.n_hidden),
                                       nn.ReLU(),
                                       nn.Dropout(dropout),
                                       nn.Linear(self.n_hidden, n_class)) # classification

    def forward(self, scalars, x, edges, node_mask, edge_mask, n_nodes):
        h = self.embedding(scalars)

        for i in range(self.n_layers):
            h, x, _ = self.LGEBs[i](h, x, edges, node_attr=scalars)

        h = h * node_mask
        h = h.view(-1, n_nodes, self.n_hidden)
        h = torch.mean(h, dim=1)
        pred = self.graph_dec(h)
        return pred.squeeze(1)

# Data preprocessing functions
def create_edges(n_particles):
    """Create fully connected edges for each particle in the jet"""
    edges = []
    for i in range(n_particles):
        for j in range(n_particles):
            if i != j:
                edges.append((i, j))
    return torch.tensor(edges).t().contiguous()

In [3]:
# Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
# Load and prepare data
train_data = pd.read_parquet('z_train.parquet').head(1000)
test_data = pd.read_parquet('z_test.parquet').head(200)

In [5]:
print(train_data.columns)
print(test_data.columns)

Index(['reco_cand_p4s', 'reco_cand_charge', 'reco_cand_pdg', 'reco_jet_p4s',
       'reco_cand_dxy', 'reco_cand_dz', 'reco_cand_dxy_err',
       'reco_cand_dz_err', 'gen_jet_p4s', 'gen_jet_tau_decaymode',
       'gen_jet_tau_p4s'],
      dtype='object')
Index(['reco_cand_p4s', 'reco_cand_charge', 'reco_cand_pdg', 'reco_jet_p4s',
       'reco_cand_dxy', 'reco_cand_dz', 'reco_cand_dxy_err',
       'reco_cand_dz_err', 'gen_jet_p4s', 'gen_jet_tau_decaymode',
       'gen_jet_tau_p4s'],
      dtype='object')


In [6]:
# 4momenta özelliklerini çıkarma
train_data_reco_jet_df = pd.DataFrame(train_data['reco_jet_p4s'].tolist())
test_data_reco_jet_df = pd.DataFrame(test_data['reco_jet_p4s'].tolist())
print(train_data_reco_jet_df)
train_data_reco_jet_df = train_data_reco_jet_df.to_numpy()
test_data_reco_jet_df = test_data_reco_jet_df.to_numpy()

             x          y           z       tau
0    -1.480275   9.609611    3.717498  0.347857
1     7.567224 -12.299459   19.494498  3.541984
2     3.145566  25.068748  -18.013098  0.139616
3    15.081155 -19.199248  -35.699724  2.340866
4   -67.428568 -36.805377   13.092938  0.797328
..         ...        ...         ...       ...
995  -8.098647 -16.194725  123.337807  0.534220
996  -6.845994  27.920566  -36.060777  7.719397
997  25.887461   5.285017   48.852423  0.953013
998 -10.641444  34.619491  -21.856480  1.314122
999  33.227768   2.197429  -43.691699  6.642142

[1000 rows x 4 columns]


In [7]:
#PDGid özelliklerini çıkarma
scalar_column='reco_cand_pdg'
max_particles=1000
train_data_pdg_df = pd.DataFrame(train_data['reco_cand_pdg'].tolist())
test_data_pdg_df = pd.DataFrame(test_data['reco_cand_pdg'].tolist())
print(train_data_pdg_df)
train_data_pdg_df = train_data_pdg_df.to_numpy()
test_data_pdg_df = test_data_pdg_df.to_numpy()

      0      1      2     3     4      5     6     7     8   9   ...  22  23  \
0    211   22.0   22.0   NaN   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
1    211   22.0  130.0  22.0   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
2    211    NaN    NaN   NaN   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
3    211   22.0   22.0  22.0   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
4    211   22.0   22.0   NaN   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
..   ...    ...    ...   ...   ...    ...   ...   ...   ...  ..  ...  ..  ..   
995   22  211.0    NaN   NaN   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
996  211  130.0    NaN   NaN   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
997   22   22.0  211.0  22.0   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
998  211  211.0   11.0  22.0   NaN    NaN   NaN   NaN   NaN NaN  ... NaN NaN   
999  211   22.0   22.0  22.0  22.0  130.0  22.0  22.0  22.0 NaN  ... NaN NaN   

     24  25  26  27  28  29  30  31  
0

In [8]:
#create tensor
train_4vecs = torch.tensor(train_data_reco_jet_df, dtype=torch.float32)
test_4vecs = torch.tensor(test_data_reco_jet_df, dtype=torch.float32)
print(train_4vecs)
train_scalars = torch.tensor(train_data_pdg_df, dtype=torch.float32)
test_scalars = torch.tensor(test_data_pdg_df, dtype=torch.float32)
print(train_scalars)

tensor([[ -1.4803,   9.6096,   3.7175,   0.3479],
        [  7.5672, -12.2995,  19.4945,   3.5420],
        [  3.1456,  25.0687, -18.0131,   0.1396],
        ...,
        [ 25.8875,   5.2850,  48.8524,   0.9530],
        [-10.6414,  34.6195, -21.8565,   1.3141],
        [ 33.2278,   2.1974, -43.6917,   6.6421]])
tensor([[211.,  22.,  22.,  ...,  nan,  nan,  nan],
        [211.,  22., 130.,  ...,  nan,  nan,  nan],
        [211.,  nan,  nan,  ...,  nan,  nan,  nan],
        ...,
        [ 22.,  22., 211.,  ...,  nan,  nan,  nan],
        [211., 211.,  11.,  ...,  nan,  nan,  nan],
        [211.,  22.,  22.,  ...,  nan,  nan,  nan]])


In [19]:
# Padding
train_scalars_padded = np.zeros((len(train_scalars), max_particles, train_scalars.shape[-1]))
test_scalars_padded = np.zeros((len(test_scalars), max_particles, test_scalars.shape[-1]))

for i, jet in enumerate(train_scalars):
    jet_len = min(len(jet), max_particles)
    train_scalars_padded[i, :jet_len, :] = jet[:jet_len]
    
for i, jet in enumerate(test_scalars):
    jet_len = min(len(jet), max_particles)
    test_scalars_padded[i, :jet_len, :] = jet[:jet_len]
    
# Create masks
train_mask = (train_scalars_padded.sum(axis=-1) != 0).astype(np.float32)
test_mask = (test_scalars_padded.sum(axis=-1) != 0).astype(np.float32)
print(train_mask)

train_masks = torch.tensor(train_mask, dtype=torch.float32)
test_masks = torch.tensor(test_mask, dtype=torch.float32)

#Label
l_train = train_data['gen_jet_tau_decaymode'].values
l_test = test_data['gen_jet_tau_decaymode'].values

# Remap labels
all_labels = np.unique(np.concatenate([l_train, l_test]))
label_map = {old: new for new, old in enumerate(all_labels)}
num_classes = len(all_labels)

l_train = np.array([label_map[label] for label in l_train])
l_test = np.array([label_map[label] for label in l_test])
print("Updated train labels:", np.unique(l_train))
print("Updated test labels:", np.unique(l_test))

print("Train Scalars Shape:", train_scalars.shape)
print("Train Vectors Shape:", train_4vecs.shape)
print("Train Masks Shape:", train_masks.shape)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
Updated train labels: [0 1 2 3 4 5 6 7 8]
Updated test labels: [0 1 2 3 5 6 7]
Train Scalars Shape: torch.Size([1000, 32])
Train Vectors Shape: torch.Size([1000, 4])
Train Masks Shape: torch.Size([1000, 1000])


In [10]:
class JetDataset(torch.utils.data.Dataset):
    def __init__(self, scalars, vectors, labels, masks, n_particles=1000):
        self.scalars = scalars
        self.vectors = vectors
        self.labels = labels
        self.masks = masks
        self.edges = create_edges(n_particles)
        self.edge_mask = create_edge_mask(self.edges)
        self.n_particles = n_particles

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (self.scalars[idx], 
                self.vectors[idx],
                self.edges,
                self.masks[idx],
                self.edge_mask,
                self.n_particles,
                self.labels[idx])

# Create datasets and dataloaders
train_dataset = JetDataset(train_scalars, train_4vecs, l_train, train_masks)
test_dataset = JetDataset(test_scalars, test_4vecs, l_test, test_masks)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

NameError: name 'create_edge_mask' is not defined

In [ ]:
# Initialize model
model = LorentzNet(
    n_scalar=train_scalars.shape[-1],
    n_hidden=64,
    n_class=num_classes,
    n_layers=6,
    c_weight=1e-3,
    dropout=0.1
).to(device)

In [ ]:
# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10

def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in tqdm(train_loader, desc='Training'):
        scalars, vectors, edges, node_mask, edge_mask, n_particles, labels = [b.to(device) for b in batch]
        
        optimizer.zero_grad()
        outputs = model(scalars, vectors, edges, node_mask, edge_mask, n_particles)
        print(outputs.shape)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(train_loader), 100. * correct / total

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            scalars, vectors, edges, node_mask, edge_mask, n_particles, labels = [b.to(device) for b in batch]
            
            outputs = model(scalars, vectors, edges, node_mask, edge_mask, n_particles)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(test_loader), 100. * correct / total

In [ ]:
# Training loop
n_epochs = 100
best_acc = 0
for epoch in range(n_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        
    print(f'Epoch: {epoch+1}/{n_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print("Best model saved!")
    
print("Training complete!")

In [ ]:
# Final Evaluation
model.load_state_dict(torch.load('best_model.pth'))
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f'Final Test Loss: {test_loss:.4f}, Final Test Acc: {test_acc:.2f}%')

In [ ]:
# Plot confusion matrix
y_true = []
y_pred = []
    
model.eval()
with torch.no_grad():
    for batch in test_loader:
        scalars, vectors, edges, node_mask, edge_mask, n_particles, labels = [b.to(device) for b in batch]
        outputs = model(scalars, vectors, edges, node_mask, edge_mask, n_particles)
        _, predicted = outputs.max(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
    
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(num_classes), yticklabels=range(num_classes))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Plot ROC Curve
y_true_bin = label_binarize(y_true, classes=range(num_classes))
y_pred_scores = []
    
with torch.no_grad():
    for batch in test_loader:
        scalars, vectors, edges, node_mask, edge_mask, n_particles, labels = [b.to(device) for b in batch]
        outputs = model(scalars, vectors, edges, node_mask, edge_mask, n_particles)
        y_pred_scores.extend(F.softmax(outputs, dim=1).cpu().numpy())
    
y_pred_scores = np.array(y_pred_scores)
    
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_scores[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()
